In [ ]:
using  GridapGmsh
using  Gridap
using  Gridap.Geometry
using  Gridap.TensorValues
using  Plots
using LinearAlgebra
using LineSearches: BackTracking
using Gridap.Arrays
using Gridap.ReferenceFEs

In [ ]:
const T0 = 300
const ΔT = 100.0
const TAppMax = T0 + ΔT

const tMax = 10.0
const n = 1000
const nExp = 50
const tTran = tMax/(n/nExp) #0.5
const Tₚ = tMax/n
const n1 = 16 # loadsteps
const delv = 0.3
const tp = Tₚ/n1
t = 0.0
const dc = 1/n1
vApp = 0.0

timev = Float64[]
AppDisplacement = Float64[]

push!(timev, t)
push!(AppDisplacement, vApp)

while  t .< tMax
    t = t + tp
    vApp = abs(4*delv/Tₚ * abs((((t-Tₚ/4)%Tₚ)+Tₚ)%Tₚ - Tₚ/2) - delv)
    #vApp = round(vAppC,digits = 6)
    push!(timev, t)
    push!(AppDisplacement, vApp)
end

In [ ]:
function Tfun(t)  
    if t <= tTran
      return ((TAppMax - T0)/tTran)*t + T0
    else
     return  TAppMax
    end
end 
TAppVec = Tfun.(timev)
plot(Tfun,0,tMax)

In [ ]:
const E = 6e3
const ν= 0.22
const Gc = 2.28
const η= 0.0
const α= 8e-6
const c = 1.899e-3
const κ_mat = 300.0
const ρ= 2450e-9

In [ ]:
## Mesh related quantities
const  L = 200.0
const  LL = 0.475.*L
const  LR = 0.525.*L
const  RS = L - 10
const  LS = 10
const  H = 40.0
const  CH = 18.0 #Crack  height
const  CW = 2.5 #Crack  Width
const  ls = 0.6
const  hf = 0.2/2.5 #Mesh  size  parameter
const  h = 30*(0.2/2.1) #Mesh  size  parameter

In [ ]:
const αT = 0.95 
const kf = 0.5
const ψ_Crit =αT * kf
const σc = sqrt(ψ_Crit*2*E)
const m = 3*E*Gc/(4*ls*σc^2)

In [ ]:
path = "./Disp_load_Results_forls$ls/"                                # Output path
mkpath(path) 
cd("Disp_load_Results_forls$ls/")

In [ ]:
using  Gmsh: gmsh

gmsh.initialize()
gmsh.option.setNumber("General.Terminal", 1)
gmsh.model.geo.addPoint((L/2)+(CW/2), 0.0 , 0.0, h ,1)
gmsh.model.geo.addPoint(RS, 0.0, 0.0, h, 101)
gmsh.model.geo.addPoint(L, 0.0, 0.0, h, 2)
gmsh.model.geo.addPoint(L, H, 0.0, h, 3)
gmsh.model.geo.addPoint(LR , H, 0.0, h, 4)
gmsh.model.geo.addPoint(LL , H, 0.0, h, 5)
gmsh.model.geo.addPoint(0.0, H, 0.0, h, 6)
gmsh.model.geo.addPoint(0.0, 0.0, 0.0, h, 7)
gmsh.model.geo.addPoint(LS, 0.0, 0.0, h, 701)
gmsh.model.geo.addPoint((L/2) -(CW/2), 0.0 , 0.0, h ,8)
gmsh.model.geo.addPoint((L/2) -(CW/2), CH-0.1*CH , 0.0, h ,801)
gmsh.model.geo.addPoint((L/2), CH , 0.0, h , 9)
gmsh.model.geo.addPoint((L/2) + (CW/2), CH-0.1*CH , 0.0, h ,901)

gmsh.model.geo.addLine(1, 101, 1)
gmsh.model.geo.addLine(101, 2, 101)
gmsh.model.geo.addLine(2, 3, 2)
gmsh.model.geo.addLine(3, 4, 3)
gmsh.model.geo.addLine(4, 5, 4)
gmsh.model.geo.addLine(5, 6, 5)
gmsh.model.geo.addLine(6, 7, 6)
gmsh.model.geo.addLine(7, 701, 701)
gmsh.model.geo.addLine(701, 8, 7)
gmsh.model.geo.addLine(8, 801, 801)
gmsh.model.geo.addLine(801, 9, 8)
gmsh.model.geo.addLine(9, 901, 901)
gmsh.model.geo.addLine(901, 1, 9)

gmsh.model.geo.addCurveLoop([1,101,2,3,4,5,6,701,7,801,8,901,9],1)
gmsh.model.geo.addPlaneSurface([1], 1)
gmsh.model.addPhysicalGroup(2, [1],1)

gmsh.model.addPhysicalGroup(1, [4],1)
gmsh.model.addPhysicalGroup(1, [6],2)
gmsh.model.addPhysicalGroup(1, [2],3)

gmsh.model.addPhysicalGroup(0, [701],2)
gmsh.model.addPhysicalGroup(0, [101],3)

gmsh.model.setPhysicalName(2, 1, "Domain")
gmsh.model.setPhysicalName(1, 1, "LoadLine")
gmsh.model.setPhysicalName(1, 2, "LeftTempBoundary")
gmsh.model.setPhysicalName(1, 3, "RightTempBoundary")

gmsh.model.setPhysicalName(0, 2, "LeftSupport")
gmsh.model.setPhysicalName(0, 3, "RightSupport")

gmsh.model.mesh.field.add("Box", 10)
gmsh.model.mesh.field.setNumber(10, "VIn", hf)
gmsh.model.mesh.field.setNumber(10, "VOut", h)
gmsh.model.mesh.field.setNumber(10, "XMin", (L/2)-4.0)
gmsh.model.mesh.field.setNumber(10, "XMax", (L/2)+4.0)
gmsh.model.mesh.field.setNumber(10, "YMin", 0)
gmsh.model.mesh.field.setNumber(10, "YMax", H)
gmsh.model.mesh.field.setAsBackgroundMesh(10)
gmsh.model.geo.synchronize()
gmsh.model.mesh.generate(2)
gmsh.write("BeamWithNotchSymThreePtBending.msh")
gmsh.finalize()

In [ ]:
model = GmshDiscreteModel("BeamWithNotchSymThreePtBending.msh")
writevtk(model ,"BeamWithNotchSymThreePtBending")

In [ ]:
function degDer(ϕ)
    g = (m*ϕ)/(0.25*m^2*ϕ^4 - 0.5*m^2*ϕ^2 + 0.25*m^2 - m*ϕ^4 + m*ϕ^2 + ϕ^4) 
    return g
end

In [ ]:
function  ElasFourthOrderConstTensor(E,ν,PlanarState)# 1 for  Plane  Stress  and 2 Plane  Strain  Condition
    if  PlanarState  == 1
        C1111 =E/(1-ν*ν)
        C1122 = (ν*E)/(1-ν*ν)
        C1112 = 0.0
        C2222 =E/(1-ν*ν)
        C2212 = 0.0
        C1212 =E/(2*(1+ν))
    elseif  PlanarState  == 2
        C1111 = (E*(1-ν*ν))/((1+ν)*(1-ν-2*ν*ν))
        C1122 = (ν*E)/(1-ν-2*ν*ν)
        C1112 = 0.0
        C2222 = (E*(1-ν))/(1-ν-2*ν*ν)
        C2212 = 0.0
        C1212 =E/(2*(1+ν))
    end
    C_ten = SymFourthOrderTensorValue(C1111 ,C1112 ,C1122 ,C1112 ,
        C1212 ,C2212 ,C1122 ,C2212 ,C2222)
    return   C_ten
end

In [ ]:
const  C_mat = ElasFourthOrderConstTensor(E,ν,1)

In [ ]:
I2 = SymTensorValue{2,Float64}(1.0 ,0.0 ,1.0)
I4 = I2 ⊗ I2
I4_sym = one(SymFourthOrderTensorValue{2,Float64})
I4_vol = (1.0/2)*I4
I4_dev = I4_sym  - I4_vol

In [ ]:
σ_elas(εElas) = C_mat ⊙ εElas

In [ ]:
function σ_elasMod(ε,ε_in , s_in ,T,T_in)
    εElas_in = ε_in - α*(T_in -T0)*I2
    εElas = ε - α*(T-T0)*I2

    gs = (s_in^2) / (s_in^2 + m*(1-s_in)*(0.5+0.5*s_in))
    
    if tr(εElas_in)  >= 0
        σ= (gs +η)*σ_elas(εElas)
    elseif  tr(εElas_in) < 0
        σ= (gs +η) *I4_dev ⊙ σ_elas(εElas) + I4_vol ⊙ σ_elas(εElas)
    end
    return σ
end

In [ ]:
function σ_totMod(ε,ε_in ,s_in, T_in)
    εElas_in = ε_in - α*(T_in -T0)*I2
    εElasTot = ε
    gs = (s_in^2) / (s_in^2 + m*(1-s_in)*(0.5+0.5*s_in))
    if tr(εElas_in) >= 0
        σU = (gs +η)*σ_elas(εElasTot)
    elseif  tr(εElas_in) < 0
        σU = (gs +η) *I4_dev ⊙ σ_elas(εElasTot) + I4_vol ⊙ σ_elas(εElasTot)
    end
    return σU
end

In [ ]:
function σ_totthMod(ε_in ,s_in, T, T_in)
    εElas_in = ε_in - α*(T_in -T0)*I2
    εElasTotth = -α*T*I2

    gs = (s_in^2) / (s_in^2 + m*(1-s_in)*(0.5+0.5*s_in))
    
    if tr(εElas_in) >= 0
        σT = (gs +η)*σ_elas(εElasTotth)
    elseif  tr(εElas_in) < 0
        σT = (gs +η) *I4_dev ⊙ σ_elas(εElasTotth) + I4_vol ⊙ σ_elas(εElasTotth)
    end
    return σT
end

In [ ]:
function σ_thermMod(ε_in ,s_in ,T_in)
    εElas_in = ε_in - α*(T_in -T0)*I2
    εElasTher = α*(T0)*I2

    gs = (s_in^2) / (s_in^2 + m*(1-s_in)*(0.5+0.5*s_in))
    
    if tr(εElas_in) >= 0
        σF = (gs +η)*σ_elas(εElasTher)
    elseif  tr(εElas_in) < 0
        σF = (gs +η)*I4_dev ⊙ σ_elas(εElasTher) + I4_vol ⊙ σ_elas(εElasTher)
    end
    return σF
end

In [ ]:
function ΨPos(ε_in ,T_in, Fdg)
    εElas_in = ε_in - α*(T_in -T0)*I2
    if tr(εElas_in)  >= 0
        ΨPlus = 0.5*((εElas_in) ⊙ σ_elas(εElas_in)) / Fdg
    elseif  tr(εElas_in) < 0
        ΨPlus = 0.5*(( I4_dev ⊙ σ_elas(εElas_in)) ⊙ (I4_dev ⊙ (εElas_in))) / Fdg
    end
    return ΨPlus
end

function ΨPosOrg(ε_in ,T_in)
    εElas_in = ε_in - α*(T_in -T0)*I2
    if tr(εElas_in)  >= 0
        ΨPlus = 0.5*((εElas_in) ⊙ σ_elas(εElas_in))
    elseif  tr(εElas_in) < 0
        ΨPlus = 0.5*(( I4_dev ⊙ σ_elas(εElas_in)) ⊙ (I4_dev ⊙ (εElas_in)))
    end
    return ΨPlus
end

In [ ]:
function αNC(s_in,ψhPos_in)
    gs = (s_in^2) / (s_in^2 + m*(1-s_in)*(0.5+0.5*s_in))
    α = gs * ψhPos_in
    return α
end

function FatiguehistoryVariable(ψhPos_in,αPrev,αbPrev)
    α = ψhPos_in #(s_in ^2 + η)*ψhPos_in
    if α >= αPrev
       αb = αbPrev + abs(α-αPrev)
    else
       αb = αbPrev
    end
return αb
end

function FatigueDegrad(αb)
    fdg = @. ifelse(αb >= αT , ((2*αT)/(αb + αT))*((2*αT)/(αb + αT)), 1.0)
    return fdg
end

In [ ]:
order = 1
reffe_PF = ReferenceFE(lagrangian ,Float64 ,order)
V0_PF = TestFESpace(model ,reffe_PF;conformity =:H1)
U_PF = TrialFESpace(V0_PF)

In [ ]:
reffe_Disp = ReferenceFE(lagrangian ,VectorValue{2,Float64},order)
V0_Disp = TestFESpace(model ,reffe_Disp;conformity =:H1 ,dirichlet_tags =["LeftSupport","RightSupport","LoadLine"],
    dirichlet_masks =[(true ,true) ,(false ,true), (false ,true)])
uh = zero(V0_Disp)

In [ ]:
reffe_Temp = ReferenceFE(lagrangian ,Float64 ,order)
V0_Temp = TestFESpace(model ,reffe_Temp;conformity =:H1 ,dirichlet_tags =["LeftTempBoundary","RightTempBoundary"])

In [ ]:
V0 = MultiFieldFESpace([V0_Disp,V0_Temp])

In [ ]:
degree = 2* order
Ω= Triangulation(model)
dΩ= Measure(Ω,degree)

In [ ]:
labels = get_face_labeling(model)
LoadTagId = get_tag_from_name(labels ,"LoadLine")
Γ_Load = BoundaryTriangulation(model ,tags = LoadTagId)
dΓ_Load = Measure(Γ_Load ,degree)
n_Γ_Load = get_normal_vector(Γ_Load)

In [ ]:
function  project(q,model ,dΩ,order)
    reffe = ReferenceFE(lagrangian ,Float64 ,order)
    V = FESpace(model ,reffe ,conformity =:L2)
    a(u,v) =∫(u*v)*dΩ
    b(v) =∫(v*q)*dΩ
    op = AffineFEOperator(a,b,V,V)
    qh = Gridap.solve(op)
    return  qh
end

In [ ]:
function crack_tip_tracker(sh,tol)
    damagetracker = Float64[]
    sVec = Float64[]
    coords = get_node_coordinates(Ω)
    
    sVec = get_free_dof_values(sh)
    
    for i in 1:length(sVec)
        if sVec[i] <= tol
            push!(damagetracker,i)
        end
    end
   damagetracker = round.(Int,damagetracker)
    xloc = Float64[]
    yloc = Float64[]
    if damagetracker!= Float64[]
        for i in 1:length(damagetracker)
            push!(xloc,coords[damagetracker[i]][1])
            push!(yloc,coords[damagetracker[i]][2])
        end
        Locy, index= findmax(yloc)
        Locx = xloc[index]
        return (Locx,Locy)
    else 
       return "No damage"
    end
end

In [ ]:
nls = NLSolver(
  show_trace=true,
  method=:newton,
  linesearch=BackTracking(), iterations = 10)
solver = FESolver(nls)

In [ ]:
function stepPhaseField(x0,vApp,cache,ψPlusPrev_in)
     U_PF = TrialFESpace(V0_PF)
     res(s,ϕ) = ∫( (3/4)*Gc*ls*∇(ϕ)⋅ ∇(s) + (degDer∘(s))*ψPlusPrev_in*ϕ - ((3/8)*Gc/ls)*ϕ)*dΩ
     op = FEOperator(res,U_PF,V0_PF)
     sh_out = FEFunction(U_PF,x0)
     sh_out, cache = solve!(sh_out,solver,op,cache)
  return sh_out, get_free_dof_values(sh_out), cache
end

In [ ]:
function κGradTemp(∇,s_in)
    gs = (s_in^2) / (s_in^2 + m*(1-s_in)*(0.5+0.5*s_in))
    kGr = (gs +η)*κ_mat*∇
    return kGr
end

In [ ]:
function    stepDispTemp(uh_in ,sh_in ,T_in ,vApp ,TApp ,delt)
    uApp1(x) = VectorValue(0.0,0.0)
    uApp2(x) = VectorValue(0.0,0.0)
    uApp3(x) = VectorValue(0.0,-vApp)
    U_Disp = TrialFESpace(V0_Disp ,[uApp1 ,uApp2, uApp3])
    
    Tapp1(x) = T0
    Tapp2(x) = TApp
    Tg = TrialFESpace(V0_Temp ,[Tapp1 , Tapp2])
    
    U = MultiFieldFESpace([U_Disp ,Tg])
    
    a((u,T) ,(v,w)) =∫( (ε(v) ⊙ (σ_totMod∘(ε(u),ε(uh_in),sh_in,T_in) + σ_totthMod∘(ε(uh_in),sh_in,T,T_in))) + ∇(w)⋅(κGradTemp∘(∇(T),sh_in)) + ((ρ*c*T*w)/delt))*dΩ
    b((v,w)) =∫(((ρ*c*T_in*w)/delt) - (ε(v)⊙(σ_thermMod∘(ε(uh_in),sh_in ,T_in))))*dΩ
    op = AffineFEOperator(a,b,U,V0)
    uhTh = Gridap.solve(op)
    uh_out ,Th_out =   uhTh
    return  uh_out ,Th_out
end

In [ ]:
function  new_EnergyState(ψPlusPrev_in ,ψhPos_in)
    ψPlus_in = ψhPos_in
    if ψPlus_in  >= ψPlusPrev_in
        ψPlus_out = ψPlus_in
    else
        ψPlus_out =ψPlusPrev_in
    end
    true ,ψPlus_out
end

In [ ]:
const tol_lim = 0.05
t = 0.0
innerMax = 10
count = 0
tol = 1e-6
cycle = 0

const dc = 1/n1

NoofCycles = Float64[]
Load = Float64[]
Displacement = Float64[]
αEnergy = Float64[]
αDegrad = Float64[]
FracEnergy = Float64[]
Xloccrack = Float64[]
Yloccrack = Float64[]

push!(NoofCycles, cycle)
push!(Load , 0.0)
push!(Displacement , 0.0)
push!(αEnergy, 0.0)
push!(αDegrad, 1.0)
push!(FracEnergy, 0.0)
push!(Xloccrack,(L/2))
push!(Yloccrack,CH)

sPrev = CellState(1.0,dΩ)
sh = project(sPrev ,model ,dΩ,order)

ThPrev = CellState(T0,dΩ)
Th = project(ThPrev ,model ,dΩ,order)

x0_PF = ones(Float64,num_free_dofs(V0_PF))
cache_1 = nothing

αPrev = CellState(0.0,dΩ)
αbPrev = CellState(0.0,dΩ)

αhPlusPrev = project(αPrev,model,dΩ,order)
αbhPlusPrev = project(αbPrev,model,dΩ,order)

FdhPrev = FatigueDegrad∘(αbhPlusPrev)
ψPlusPrev = CellState(ψ_Crit,dΩ)

while t .< tMax
    count = count  .+ 1
    t = t + tp
    cycle = cycle + dc
    vApp = AppDisplacement[count]
    TApp = TAppVec[count]

    FdhPrev = FatigueDegrad∘(αbhPlusPrev)
    
    print("\n Entering  displacemtent  step $count :", float(vApp))
    for  inner = 1:innerMax
        ψhPlusPrev = project(ψPlusPrev ,model ,dΩ,order)
        
        RelErr = abs(sum(∫( (3/4)*Gc*ls*∇(sh)⋅ ∇(sh) + (degDer∘(sh))*ψhPlusPrev*sh)*dΩ-∫( ((3/8)*Gc/ls)*sh)*dΩ))/abs(sum(∫( ((3/8)*Gc/ls)*sh)*dΩ))
        print("\n Relative error = ",float(RelErr))
        
        sh,x0_PF,cache_1 = stepPhaseField(x0_PF,vApp,cache_1,ψhPlusPrev)
        uh ,Th = stepDispTemp(uh,sh ,Th,vApp ,TApp ,tp)
        ΨhPos_in =ΨPos∘(ε(uh),Th,FdhPrev)
        update_state!(new_EnergyState ,ψPlusPrev ,ΨhPos_in)
        
        if  RelErr  < tol 
            break
        end
    end

    if  crack_tip_tracker(sh,tol_lim) =="No damage"
        push!(Xloccrack,0.5*L)
        push!(Yloccrack,CH)
    else
        (X_t, Y_t) = crack_tip_tracker(sh,tol_lim)
        push!(Xloccrack,X_t)
        push!(Yloccrack,Y_t)
    end

    ψhPosOrg_in = ΨPosOrg∘(ε(uh),Th)
    αPrev = αNC∘(sh,ψhPosOrg_in)
    αbPrev = FatiguehistoryVariable∘(αPrev,αhPlusPrev,αbhPlusPrev)
    
    αhPlusPrev = project(αPrev,model,dΩ,order)
    αbhPlusPrev = project(αbPrev,model,dΩ,order)
    
    FdhPrev = FatigueDegrad∘(αbhPlusPrev)
    
    FracEnergyExp = sum(∫((3/8)*((1-sh)/ls + ls*(∇(sh) ⋅ ∇(sh))))*dΩ)
    
    αbPrevVal = evaluate(αbPrev,VectorValue(0.5*L, 0.6*H))
    Fdegrad = evaluate(FdhPrev,VectorValue(0.5*L, 0.6*H))
    
    push!(NoofCycles, cycle)
    
    push!(αEnergy, αbPrevVal)
    push!(αDegrad, Fdegrad)
    push!(FracEnergy, FracEnergyExp)
    
    Node_Force = sum(∫( n_Γ_Load⋅(σ_elasMod∘(ε(uh),ε(uh),sh,Th,Th)) ) *dΓ_Load)
    push!(Load , Node_Force[2])
    push!( Displacement , vApp)
    if mod(count,10) == 0
    writevtk(Ω,"results_PhaseFieldThermoElastic$count",cellfields=["uh"=>uh,"s"=>sh ,"epsi"=>ε(uh),"T"=>Th])
    end
end

In [ ]:
plot(Displacement ,Load)

In [ ]:
plot(NoofCycles ,Displacement)

In [ ]:
plot(NoofCycles ,Yloccrack.-CH)

In [ ]:
using DelimitedFiles
using CSV
using LaTeXStrings

In [ ]:
path = "../PlottingFiles/"                                # Output path
mkpath(path) 
cd("../PlottingFiles/")

In [ ]:
NoofCyclesCSV = writedlm("kf05_Step64NoofCyclesPhlsfullVolDevN$ls.csv",  NoofCycles, ',')
XloccrackCSV = writedlm("kf05_Step64XloccrackPhlsfullVolDevN$ls.csv",  Xloccrack, ',')
YloccrackCSV = writedlm("kf05_Step64YloccrackPhlsfullVolDevN$ls.csv",  Yloccrack, ',')
LoadCSV = writedlm("kf05_Step64LoadPhlsfullVolDevN$ls.csv",  Load, ',')
αDegradCSV = writedlm("kf05_Step64αDegradPhlsfullVolDevN$ls.csv",  αDegrad, ',')
FracEnergyCSV = writedlm("kf05_Step64FracEnergyPhlsfullVolDevNew$ls.csv",  FracEnergy, ',')